In [1]:

%cd ..


/data2/var/www/hawk-fast-api


/data2/var/www/hawk-fast-api/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from databases.pgdb import PgDB

with PgDB() as db:
    sql_query = """
    SELECT * FROM dnb.daily_stats.dc_e_nr_events_nrcelldu_standard_sa_day t1
    INNER JOIN dnb.daily_stats.dc_e_nr_events_nrcellcu_standard_sa_day t2
        on t1.date_id = t2.date_id
        and t1.nrcelldu = t2.nrcellcu
    INNER JOIN dnb.daily_stats.dc_e_nr_events_nrcellcu_v_standard_sa_day t3
        on t1.date_id = t3.date_id
        and t1.nrcelldu = t3.nrcellcu
    INNER JOIN dnb.daily_stats.dl_prb_sa_nsa_standard_extended_day t4
        on t1.date_id = t4.date_id
        and t1.nrcelldu = t4.nrcelldu
    order by random()
    LIMIT 1000
    """
    df = db.query_df(sql_query)

df.describe()

,mean_user_dl_throughput_nr_nom,mean_user_dl_throughput_nr_den,mean_user_ul_throughput_nr_nom,mean_user_ul_throughput_nr_den,uplink_traffic_volume_nr_gb_nom,uplink_traffic_volume_nr_gb_den,downlink_traffic_volume_nr_gb_nom,downlink_traffic_volume_nr_gb_den,modulation_dl_den,dl_qpsk_nom,...,vonr_drop_call_rate_nom,vonr_drop_call_rate_den,sa_data_session_setup_success_rate_nom,sa_data_session_setup_success_rate_den,sa_data_session_abnormal_release_nom,sa_data_session_abnormal_release_den,dl_prb_utilization_nom,dl_prb_utilization_den,ul_prb_utilization_nom,ul_prb_utilization_den
count,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03,1000.0,1000.0,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03
mean,8.452917e+10,3.506888e+08,1.557823e+10,3.575912e+08,2.808125e+08,1.073742e+09,2.918797e+09,1.073742e+09,0.0,0.0,...,0.057000,3.235000,90.911000,91.009000,0.259000,101.764000,8.097855e+16,1.531976e+17,3.025886e+16,5.183292e+16
std,3.427841e+11,1.036547e+09,8.144272e+10,1.446859e+09,1.601515e+09,0.000000e+00,1.668104e+10,0.000000e+00,0.0,0.0,...,0.763108,18.406373,604.645936,604.714871,1.498721,606.010114,9.635125e+16,1.095346e+17,3.773119e+16,2.157403e+16
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.073742e+09,0.000000e+00,1.073742e+09,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.073742e+09,2.227425e+04,1.073742e+09,0.0,0.0,...,0.000000,0.000000,1.000000,1.000000,0.000000,2.000000,1.757025e+15,2.524312e+16,3.873697e+12,4.598289e+16
50%,2.620032e+06,2.400000e+05,1.418400e+06,4.920000e+05,2.866650e+04,1.073742e+09,9.738470e+05,1.073742e+09,0.0,0.0,...,0.000000,0.000000,2.000000,2.000000,0.000000,6.000000,4.979298e+16,2.239930e+17,2.133742e+16,6.085472e+16
75%,1.462405e+10,1.743970e+08,2.252356e+08,6.070000e+07,3.823670e+06,1.073742e+09,2.612218e+08,1.073742e+09,0.0,0.0,...,0.000000,0.000000,13.000000,13.000000,0.000000,24.000000,1.323427e+17,2.542143e+17,5.106501e+16,6.844230e+16
max,6.574398e+12,1.362771e+10,1.343617e+12,2.199574e+10,2.943927e+10,1.073742e+09,2.152821e+11,1.073742e+09,0.0,0.0,...,20.000000,466.000000,9139.000000,9139.000000,26.000000,9140.000000,6.997964e+17,2.543461e+17,5.662024e+17,6.847779e+16


In [13]:
(
    df.describe().T
    .loc[:, ['mean', 'std', 'min', 'max']]
    .assign(
        min_max_diff = lambda x: x['max'] - x['min'],
    )
    .loc[lambda x: x['min_max_diff'] < 0.01]
)

,mean,std,min,max,min_max_diff
uplink_traffic_volume_nr_gb_den,1.073742e+09,0.0,1.073742e+09,1.073742e+09,0.0
downlink_traffic_volume_nr_gb_den,1.073742e+09,0.0,1.073742e+09,1.073742e+09,0.0
modulation_dl_den,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
dl_qpsk_nom,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
dl_16qam_nom,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
dl_64qam_nom,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
dl_256qam_nom,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
rrc_resume_sr_nom,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
rrc_resume_sr_den,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0
total_traffic_volume_gb_den,1.073742e+09,0.0,1.073742e+09,1.073742e+09,0.0
